In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import os
from sklearn.metrics import accuracy_score, classification_report

# 禁用科學記號表示
np.set_printoptions(suppress=True)

# 載入模型
model = load_model("0408keras_Model.h5", compile=False)

# 載入標籤
class_names_from_file = open("labels.txt", "r").readlines()
class_names_list = [name.strip() for name in class_names_from_file]
num_classes = len(class_names_list)
class_to_index = {name: i for i, name in enumerate(class_names_list)}

# 測試圖片資料夾路徑
test_folder = r"E:\Codes\CV train\回收系統dataset\trashbox"  # 使用原始字串以避免反斜線問題

# 用於儲存真實標籤和預測結果的列表
true_labels = []
predicted_classes = []

# 創建正確形狀的陣列
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

for class_folder in os.listdir(test_folder):
    class_path = os.path.join(test_folder, class_folder)
    if os.path.isdir(class_path):  # 確保是資料夾
        if class_folder in class_to_index:
            true_label_index = class_to_index[class_folder]
            for filename in os.listdir(class_path):
                if filename.endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(class_path, filename)
                    try:
                        # 載入圖片
                        image = Image.open(image_path).convert("RGB")
                        size = (224, 224)
                        image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
                        image_array = np.asarray(image)
                        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
                        data[0] = normalized_image_array

                        # 進行預測
                        prediction = model.predict(data)
                        index = np.argmax(prediction)
                        predicted_class_index = index
                        predicted_classes.append(predicted_class_index)
                        true_labels.append(true_label_index)
                    except Exception as e:
                        print(f"處理圖片 {filename} 時發生錯誤：{e}")
                        print("-" * 20)
        else:
            print(f"警告：資料夾名稱 '{class_folder}' 在 labels.txt 中找不到。")

# 確保我們有真實標籤和預測結果
if true_labels and predicted_classes and len(true_labels) == len(predicted_classes):
    # 計算準確率
    accuracy = accuracy_score(true_labels, predicted_classes)
    print(f"準確率: {accuracy:.4f}")

    # 顯示詳細分類報告
    print(classification_report(true_labels, predicted_classes, target_names=class_names_list, digits=4))
else:
    print("警告：無法計算準確率和分類報告，因為真實標籤或預測結果為空或長度不一致。")

準確率: 0.6782
              precision    recall  f1-score   support

          BG     0.8144    1.0000    0.8977       373
       metal     0.4656    0.3949    0.4273      1593
       paper     0.6905    0.8800    0.7738      4066
          PG     0.7362    0.5471    0.6277      3734

    accuracy                         0.6782      9766
   macro avg     0.6767    0.7055    0.6816      9766
weighted avg     0.6760    0.6782    0.6662      9766

